In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from torch.utils.data import Dataset
import torchvision
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.nn import Conv2d
from torchvision.transforms import Compose
from torchvision.transforms import Resize
from torchvision.transforms import CenterCrop
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor
import numpy as np
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
import os
from torch.utils.data import random_split

In [ ]:
# SAANVI COMPUTER CODE
data = '/content/drive/MyDrive/UCSB SRA Project/TrainingData_Final'
print()
dirs = os.listdir(data)
dataset_old = []
for dir in dirs:
  for inner_dir in os.listdir(os.path.join(data, dir)):
      dataset_old.append(os.path.join(data, dir, inner_dir))
# print(dataset_old)
dataset = []
curr = 0
for i in dataset_old:
  if i[59] != '5':
    dataset.append(i)
  elif curr < 400:
    dataset.append(i)
    curr += 1
#59nd index
# print(len(dataset), curr)

In [ ]:
# SAANVI COMPUTER CODE
train_set = []
test_set = []
validation_set = []
for i in range(0, 6):
  path = f"/content/drive/MyDrive/UCSB SRA Project/TrainingData_Final/{i}"
  all_imgs_0 = os.listdir(path)
  # /content/drive/MyDrive/TrainingData_Final/0
  all_imgs_0 = [f"{path}/{_path}" for _path in all_imgs_0]
  # print(all_imgs_0)
  if i == 5:
    all_imgs_0 = all_imgs_0[:400]
    # print(len(all_imgs_0))
  num_imgs_0 = len(all_imgs_0)
  arr = np.arange(num_imgs_0)
  np.random.shuffle(arr)
  # print(i, 'total:', num_imgs_0)
  curr_tr = len(train_set)
  curr_te = len(test_set)
  curr_va = len(validation_set)
  train_set += [all_imgs_0[idx] for idx in arr[0:int(0.8*num_imgs_0)]]
  test_set += [all_imgs_0[idx]  for idx in arr[int(0.8*num_imgs_0):int(0.9*num_imgs_0)]]
  validation_set += [all_imgs_0[idx]  for idx in arr[int(0.9*num_imgs_0):int(num_imgs_0)]]
  print(i, 'train: ', len(train_set) - curr_tr)
  print(i, 'test: ', len(test_set) - curr_te)
  print(i, 'val: ', len(validation_set) - curr_va)
  print('---')
print('total:', len(train_set) + len(test_set) + len(validation_set))
print('train:', len(train_set))
print('test:', len(test_set))
print('val:', len(validation_set))

0 train:  439
0 test:  55
0 val:  55
---
1 train:  796
1 test:  99
1 val:  100
---
2 train:  128
2 test:  16
2 val:  17
---
3 train:  153
3 test:  19
3 val:  20
---
4 train:  28
4 test:  3
4 val:  4
---
5 train:  320
5 test:  40
5 val:  40
---
total: 2332
train: 1864
test: 232
val: 236


In [ ]:
# SAANVI COMPUTER CODE
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

labels = []
class CustomDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize the image to (256, 256)
            transforms.ToTensor(),          # Convert the PIL image to a PyTorch tensor
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the tensor
        ])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        image = Image.open(file_path)

        if self.transform:
            image = self.transform(image)


        return image, file_path[59]
train_dataloader = DataLoader(CustomDataset(train_set), batch_size=64, shuffle=True)
test_dataloader = DataLoader(CustomDataset(test_set), batch_size=64, shuffle=True)
val_dataloader = DataLoader(CustomDataset(validation_set), batch_size=64, shuffle=True)

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [ ]:
train_features, train_labels = next(iter(train_dataloader))
img = train_features[0].squeeze()
label = train_labels[0]

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, block, layers, num_classes = 5):
        super(NeuralNetwork, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(2048, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
class NeuralNetwork2(nn.Module):
    def __init__(self, block, layers, num_classes = 2):
        super(NeuralNetwork2, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(2048, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
'''class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.linear_relu_stack = nn.Sequential(
      nn.Conv2d(3, 4, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(4, 8, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(8, 16, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Flatten(),
      nn.Linear(14400, 5000),
      nn.Linear(5000, 5), #nn.Linear(10000, 6)
      nn.Softmax(dim=1)
    )
  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits'''

'class NeuralNetwork(nn.Module):\n  def __init__(self):\n    super(NeuralNetwork, self).__init__()\n    self.linear_relu_stack = nn.Sequential(\n      nn.Conv2d(3, 4, kernel_size=3),\n      nn.ReLU(),\n      nn.MaxPool2d(kernel_size=2, stride = 2),\n      nn.Conv2d(4, 8, kernel_size=3),\n      nn.ReLU(),\n      nn.MaxPool2d(kernel_size=2, stride = 2),\n      nn.Conv2d(8, 16, kernel_size=3),\n      nn.ReLU(),\n      nn.MaxPool2d(kernel_size=2, stride = 2),\n      nn.Flatten(),\n      nn.Linear(14400, 5000),\n      nn.Linear(5000, 5), #nn.Linear(10000, 6)\n      nn.Softmax(dim=1)\n    )\n  def forward(self, x):\n    logits = self.linear_relu_stack(x)\n    return logits'

In [ ]:
'''class NeuralNetwork2(nn.Module):
  def __init__(self):
    super(NeuralNetwork2, self).__init__()
    self.linear_relu_stack = nn.Sequential(
      nn.Conv2d(3, 4, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(4, 8, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(8, 16, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Flatten(),
      nn.Linear(14400, 5000),
      nn.Linear(5000, 2), #nn.Linear(10000, 6)
      # nn.Sigmoid(),
      # nn.softmax
    )
  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits'''

'class NeuralNetwork2(nn.Module):\n  def __init__(self):\n    super(NeuralNetwork2, self).__init__()\n    self.linear_relu_stack = nn.Sequential(\n      nn.Conv2d(3, 4, kernel_size=3),\n      nn.ReLU(),\n      nn.MaxPool2d(kernel_size=2, stride = 2),\n      nn.Conv2d(4, 8, kernel_size=3),\n      nn.ReLU(),\n      nn.MaxPool2d(kernel_size=2, stride = 2),\n      nn.Conv2d(8, 16, kernel_size=3),\n      nn.ReLU(),\n      nn.MaxPool2d(kernel_size=2, stride = 2),\n      nn.Flatten(),\n      nn.Linear(14400, 5000),\n      nn.Linear(5000, 2), #nn.Linear(10000, 6)\n      # nn.Sigmoid(),\n      # nn.softmax\n    )\n  def forward(self, x):\n    logits = self.linear_relu_stack(x)\n    return logits'

In [ ]:
def label_changer(y):
  for i in range(len(y)):
    #type(y)
    y[i] = int(y[i])
    if y[i] == 3 or y[i] == 5:
      y[i] = 0
    elif y[i] != 4:
      y[i] += 1
    num = y[i]
    y[i] = torch.tensor([[0, 0, 0, 0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  return y
from re import I
model = NeuralNetwork(ResidualBlock, [3, 4, 6, 3]).to(device)
batch_size = 64
# epochs = 5
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    optimizer.zero_grad()
  # Compute prediction and loss
    X = X.cpu()
    #y.expand(-1, 4)
    y = list(y)
    y = label_changer(y)
    y = y.cpu()
    pred = model(X)
    pred = pred.to(torch.float32)
    y = y.to(torch.float32)
    # print(y.shape)
    loss = loss_fn(pred, y.argmax(1))
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    # print(y)
    # print(pred)
    # print(pred.argmax(1))
    # print('pred', pred)
    # print('y', y)
    accuracy = (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    print(accuracy)
    # print('---')
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for batch, (X, y) in enumerate(dataloader):
      X = X.cpu()
      y = list(y)
      y = label_changer(y)
      y = y.cpu()
      pred = model(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 20
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


KeyboardInterrupt: ignored

In [ ]:
# label 0: Mild Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 0:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2(ResidualBlock, [3, 4, 6, 3]).to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
      print(pred.argmax(1))
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.611116 [    0/ 1864]
loss: 6.981918 [   64/ 1864]
loss: 0.790580 [  128/ 1864]
loss: 0.780735 [  192/ 1864]
loss: 0.852717 [  256/ 1864]
loss: 0.699635 [  320/ 1864]
loss: 0.780472 [  384/ 1864]
loss: 1.374881 [  448/ 1864]
loss: 1.648653 [  512/ 1864]
loss: 2.419097 [  576/ 1864]
loss: 1.701950 [  640/ 1864]
loss: 1.240710 [  704/ 1864]
loss: 0.634931 [  768/ 1864]
loss: 0.559811 [  832/ 1864]
loss: 0.660068 [  896/ 1864]
loss: 0.759758 [  960/ 1864]
loss: 0.821238 [ 1024/ 1864]
loss: 0.767732 [ 1088/ 1864]
loss: 0.722568 [ 1152/ 1864]
loss: 0.607328 [ 1216/ 1864]
loss: 0.479482 [ 1280/ 1864]
loss: 0.936873 [ 1344/ 1864]
loss: 0.780490 [ 1408/ 1864]
loss: 0.978710 [ 1472/ 1864]
loss: 0.550032 [ 1536/ 1864]
loss: 0.569752 [ 1600/ 1864]
loss: 0.584130 [ 1664/ 1864]
loss: 0.825615 [ 1728/ 1864]
loss: 0.597856 [ 1792/ 1864]
loss: 0.168373 [  232/ 1864]
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,

In [ ]:
torch.save(model2.state_dict(),  "binary0model")

In [ ]:
# label 1: Moderate Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 1:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2(ResidualBlock, [3, 4, 6, 3]).to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.741083 [    0/ 1864]
loss: 22.239538 [   64/ 1864]
loss: 2.054401 [  128/ 1864]
loss: 1.492692 [  192/ 1864]
loss: 2.272995 [  256/ 1864]
loss: 2.091600 [  320/ 1864]
loss: 4.342196 [  384/ 1864]
loss: 3.786101 [  448/ 1864]
loss: 0.965240 [  512/ 1864]
loss: 0.895054 [  576/ 1864]
loss: 2.357228 [  640/ 1864]
loss: 4.085869 [  704/ 1864]
loss: 4.566681 [  768/ 1864]
loss: 2.068973 [  832/ 1864]
loss: 1.619917 [  896/ 1864]
loss: 0.702635 [  960/ 1864]
loss: 1.088595 [ 1024/ 1864]
loss: 0.696505 [ 1088/ 1864]
loss: 2.071493 [ 1152/ 1864]
loss: 2.775524 [ 1216/ 1864]
loss: 0.913383 [ 1280/ 1864]
loss: 1.939711 [ 1344/ 1864]
loss: 1.935739 [ 1408/ 1864]
loss: 1.907807 [ 1472/ 1864]
loss: 0.693241 [ 1536/ 1864]
loss: 1.315539 [ 1600/ 1864]
loss: 0.694579 [ 1664/ 1864]
loss: 0.676846 [ 1728/ 1864]
loss: 1.261357 [ 1792/ 1864]
loss: 2.453486 [  232/ 1864]
Test Error: 
 Accuracy: 47.8%, Avg loss: 0.021776 

Epoch 2
-----------------------------

In [ ]:
torch.save(model2.state_dict(),  "binary1model")

In [ ]:
# label 2: Severe Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 2:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2(ResidualBlock, [3, 4, 6, 3]).to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.939327 [    0/ 1864]
loss: 0.605872 [   64/ 1864]
loss: 0.081005 [  128/ 1864]
loss: 0.700599 [  192/ 1864]
loss: 0.764113 [  256/ 1864]
loss: 0.434182 [  320/ 1864]
loss: 0.479547 [  384/ 1864]
loss: 0.175173 [  448/ 1864]
loss: 0.127433 [  512/ 1864]
loss: 1.398418 [  576/ 1864]
loss: 2.151008 [  640/ 1864]
loss: 2.876752 [  704/ 1864]
loss: 3.568269 [  768/ 1864]
loss: 0.423833 [  832/ 1864]
loss: 12.245138 [  896/ 1864]
loss: 0.404482 [  960/ 1864]
loss: 0.286029 [ 1024/ 1864]
loss: 0.176831 [ 1088/ 1864]
loss: 10.531519 [ 1152/ 1864]
loss: 0.345751 [ 1216/ 1864]
loss: 0.219497 [ 1280/ 1864]
loss: 0.350072 [ 1344/ 1864]
loss: 7.030770 [ 1408/ 1864]
loss: 0.200916 [ 1472/ 1864]
loss: 16.686541 [ 1536/ 1864]
loss: 0.506054 [ 1600/ 1864]
loss: 0.372632 [ 1664/ 1864]
loss: 1.227910 [ 1728/ 1864]
loss: 0.227851 [ 1792/ 1864]
loss: 1.176767 [  232/ 1864]
Test Error: 
 Accuracy: 93.1%, Avg loss: 0.012088 

Epoch 2
---------------------------

In [ ]:
torch.save(model2.state_dict(),  "binary2model")

In [ ]:
# label 3: Hypertensive  Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 3:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2(ResidualBlock, [3, 4, 6, 3]).to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.603056 [    0/ 1864]
loss: 3.309623 [   64/ 1864]
loss: 0.180033 [  128/ 1864]
loss: 0.451704 [  192/ 1864]
loss: 0.707533 [  256/ 1864]
loss: 0.566629 [  320/ 1864]
loss: 0.359512 [  384/ 1864]
loss: 0.263396 [  448/ 1864]
loss: 0.483336 [  512/ 1864]
loss: 0.310663 [  576/ 1864]
loss: 0.182248 [  640/ 1864]
loss: 0.288021 [  704/ 1864]
loss: 0.677274 [  768/ 1864]
loss: 0.438520 [  832/ 1864]
loss: 0.343908 [  896/ 1864]
loss: 0.456282 [  960/ 1864]
loss: 0.310718 [ 1024/ 1864]
loss: 0.390299 [ 1088/ 1864]
loss: 1.707536 [ 1152/ 1864]
loss: 0.305196 [ 1216/ 1864]
loss: 0.204378 [ 1280/ 1864]
loss: 0.374779 [ 1344/ 1864]
loss: 1.329537 [ 1408/ 1864]
loss: 0.310541 [ 1472/ 1864]
loss: 0.378168 [ 1536/ 1864]
loss: 1.719177 [ 1600/ 1864]
loss: 3.734402 [ 1664/ 1864]
loss: 17.483799 [ 1728/ 1864]
loss: 3.801965 [ 1792/ 1864]
loss: 2.926520 [  232/ 1864]
Test Error: 
 Accuracy: 91.8%, Avg loss: 0.016085 

Epoch 2
-----------------------------

In [ ]:
torch.save(model2.state_dict(),  "binary3model")

In [ ]:
# label 4: Severe Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 4:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2(ResidualBlock, [3, 4, 6, 3]).to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.653616 [    0/ 1864]
loss: 0.000003 [   64/ 1864]
loss: 1.666794 [  128/ 1864]
loss: 0.000000 [  192/ 1864]
loss: 0.000000 [  256/ 1864]
loss: 0.000000 [  320/ 1864]
loss: 0.584355 [  384/ 1864]
loss: 0.692171 [  448/ 1864]
loss: 0.830454 [  512/ 1864]
loss: 0.000001 [  576/ 1864]
loss: 0.227490 [  640/ 1864]
loss: 0.152749 [  704/ 1864]
loss: 0.097274 [  768/ 1864]
loss: 0.405761 [  832/ 1864]
loss: 0.031809 [  896/ 1864]
loss: 0.099928 [  960/ 1864]
loss: 0.344418 [ 1024/ 1864]
loss: 0.182164 [ 1088/ 1864]
loss: 0.017796 [ 1152/ 1864]
loss: 0.306815 [ 1216/ 1864]
loss: 0.001012 [ 1280/ 1864]
loss: 0.006450 [ 1344/ 1864]
loss: 0.078558 [ 1408/ 1864]
loss: 0.006148 [ 1472/ 1864]
loss: 0.081165 [ 1536/ 1864]
loss: 0.166832 [ 1600/ 1864]
loss: 5.408880 [ 1664/ 1864]
loss: 0.009323 [ 1728/ 1864]
loss: 0.013250 [ 1792/ 1864]
loss: 0.000033 [  232/ 1864]
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.002555 

Epoch 2
------------------------------

KeyboardInterrupt: ignored

In [ ]:
torch.save(model2.state_dict(),  "binary4model")